In [13]:
from utils.config import get_config
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model
from database.vector_store.vector_store import ElasticIndexManager, RedisIndexManager
from database.vector_store.neo4j_graph_store import Neo4jGraphStore
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo
from database.chat_store import ElasticChatStore
from chain.rag_chain import create_combined_answer_chain, create_sequential_chain, create_combined_context_chain
from chain.chain_sikepo.graph_cypher_sikepo_chain import graph_rag_chain
from chain.rag_chain import get_response
from typing import AsyncGenerator
import time
from typing import Union
from fastapi import Query

config = get_config()

# =========== GLOBAL VARIABLES ===========
llm_model, embed_model = get_model(model_name=ModelName.AZURE_OPENAI, config=config,
                                   llm_model_name=LLMModelName.GPT_AZURE, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)
top_k = 4

# index_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
# index_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
# index_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
# index_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
index_ojk = ElasticIndexManager(
    index_name='ojk', embed_model=embed_model, config=config)
index_bi = ElasticIndexManager(
    index_name='bi', embed_model=embed_model, config=config)
index_sikepo_ket = ElasticIndexManager(
    index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config)
index_sikepo_rek = ElasticIndexManager(
    index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config)

vector_store_ojk = index_ojk.load_vector_index()
vector_store_bi = index_bi.load_vector_index()
vector_store_ket = index_sikepo_ket.load_vector_index()
vector_store_rek = index_sikepo_rek.load_vector_index()

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

# chat_store = RedisChatStore(k=3, config=config)
chat_store = ElasticChatStore(k=4, config=config)

# Initialize retrievers and chains with default model
retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_k=top_k,
                                  llm_model=llm_model, embed_model=embed_model, config=config)
retriever_ojk_wo_self = get_retriever_ojk(vector_store=vector_store_ojk, top_k=top_k,
                                  llm_model=llm_model, embed_model=embed_model, config=config, with_self_query=False)
retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_k=top_k,
                                llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, top_k=top_k,
                                   llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket_wo_self = lotr_sikepo(vector_store=vector_store_ket, top_k=top_k,
                                   llm_model=llm_model, embed_model=embed_model, config=config, with_self_query=False)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, top_k=top_k,
                                   llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek_wo_self = lotr_sikepo(vector_store=vector_store_rek, top_k=top_k,
                                   llm_model=llm_model, embed_model=embed_model, config=config, with_self_query=False)

graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)
chain = create_combined_context_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_bi,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)
chain_wo_self = create_combined_context_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk_wo_self,
    retriever_bi=retriever_bi,
    retriever_sikepo_ketentuan=retriever_sikepo_ket_wo_self,
    retriever_sikepo_rekam=retriever_sikepo_rek_wo_self,
)

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://16.78.85.84:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


Loaded index 'ojk'.
Loaded index 'bi'.
Loaded index 'sikepo-ketentuan-terkait'.
Loaded index 'sikepo-rekam-jejak'.


Failed to write data to connection IPv4Address(('108.137.122.156', 7687)) (ResolvedIPv4Address(('108.137.122.156', 7687)))


In [14]:
chain.invoke({"question":"Siapa yang bisa memiliki Bank?", "chat_history":""})

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

{'rewrited question': 'Siapa yang dapat memiliki sebuah Bank?',
 'context': [Document(metadata={'doc_id': 2680, 'title': 'Surat Edaran Otoritas Jasa Keuangan Nomor 12/SEOJK.03/2017', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '12/SEOJK.03/2017', 'effective_date': '2017/03/17', 'file_url': 'https://www.ojk.go.id/id/kanal/perbankan/regulasi/surat-edaran-ojk/Documents/SAL%20SEOJK%2012%20%20Kepemilikan%20Saham%20BU.pdf', 'page_number': 1}, page_content='Bank dalam menerapkan Tata Kelola yang baik. Pengalaman krisis pada \nmasa lalu membuktikan bahwa Bank yang terkena dampak krisis adalah \nBank yang dimiliki secara dominan oleh pemegang saham tertentu. Oleh \nkarena itu, perlu dilakukan penyebaran kepemilikan saham Bank dengan \nmenerapkan batas maksimum kepemilikan saham Bank sehingga Bank \ndapat menerapkan Tata Kelola yang baik. \n \nII. \nPENERAPAN BATAS MAKSIMUM KEPEMILIKAN SAHAM BANK  \nA. Calon Pemegang Saham Bank  \n1. Calon pemegang saham dapa

In [15]:
chain_wo_self.invoke({"question":"Siapa yang bisa memiliki Bank?", "chat_history":""})

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'rewrited question': 'Siapa yang dapat memiliki sebuah Bank?',
 'context': [Document(metadata={'doc_id': 2562, 'title': 'Peraturan Bank Indonesia Nomor 6/24/PBI/2004', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '6/24/PBI/2004', 'effective_date': '2004/10/14', 'file_url': 'https://www.ojk.go.id/id/kanal/perbankan/regulasi/peraturan-bank-indonesia/Documents/317.pdf', 'page_number': 19}, page_content='- 19 - \n(3) Selain... \nwajib dipenuhi pada saat badan hukum yang bersangkutan melakukan \npenyetoran modal untuk pendirian Bank atau pada saat badan hukum \nyang bersangkutan melakukan penambahan modal disetor Bank. \n \nPasal 14 \nSumber dana yang digunakan dalam rangka kepemilikan Bank dilarang:  \na. berasal dari pinjaman atau fasilitas pembiayaan dalam bentuk apapun \ndari bank dan/atau pihak lain; dan atau \nb. berasal dari sumber yang diharamkan menurut prinsip syariah, termasuk \ndari dan untuk tujuan pencucian uang (money laundering). \n \nPas

In [5]:
chain.invoke({"question":"Siapa yang bisa memiliki Bank?", "chat_history":""})

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

{'rewrited question': 'Siapa yang dapat memiliki sebuah Bank?',
 'context': [Document(metadata={'doc_id': 2009, 'title': 'Surat Edaran Bank Indonesia Nomor 15/4/DPNP', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '15/4/DPNP/2013', 'effective_date': '2013/03/06', 'file_url': 'https://www.ojk.go.id/Files/regulasi/perbankan/se-bi/2013/faq_se_150413.pdf', 'page_number': 1}, page_content='dapat membeli saham bank tersebut di atas batas maksimum kepemilikan saham\natas maksimum kepemilikan saham\natas maksimum kepemilikan saham\natas maksimum kepemilikan saham?\n?\n?\n? \n \n \n \nTidak dapat, calon pemegang saham hanya dapat memiliki saham Bank paling tinggi sebesar \nbatas maksimum kepemilikan saham pada saat yang bersangkutan menjadi pemegang saham \nBank dimaksud. \n2.\n2.\n2.\n2. Bagaimana dengan batas maksimum kepemilikan saham bagi \nBagaimana dengan batas maksimum kepemilikan saham bagi \nBagaimana dengan batas maksimum kepemilikan saham bagi \nBag

In [4]:
llm_model

AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x00000201E83A5160>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000201E83B02E0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://crayonchatbot2.openai.azure.com/', deployment_name='gpt-4o-mini', openai_api_version='2024-06-01', openai_api_type='openai')

In [3]:
llm_model.invoke("Who are you?")

AIMessage(content="I'm an AI language model created by OpenAI, designed to assist with a wide range of questions and tasks. How can I help you today?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 11, 'total_tokens': 40}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_276aa25277', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-2e6620dd-2b91-4294-bcb8-276c75084ce5-0', usage_metadata

In [12]:
llm_model.invoke("Who are you?")

AIMessage(content="I'm an AI language model created by OpenAI, designed to assist with a wide range of questions and tasks. How can I help you today?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 11, 'total_tokens': 40}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_276aa25277', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-87e97a73-625b-4c3a-8464-54812b63548a-0', usage_metadata

In [2]:
from langchain.retrievers import MergerRetriever, ContextualCompressionRetriever
from retriever.retriever_sikepo.self_query_sikepo import self_query_retriever_sikepo
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from langchain_community.document_transformers import (
    EmbeddingsRedundantFilter, LongContextReorder,
)
from langchain_community.document_transformers.embeddings_redundant_filter import _DocumentWithState
# from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank
# from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank
from langchain.retrievers.document_compressors.base import DocumentCompressorPipeline

def unique_context(context: list):
    context_unique = [i for n, i in enumerate(context) if i not in context[:n]]
    return context_unique

def unique_context_2(context: list[Document]):
    context_unique = []
    for i in context:
        if len(context_unique) == 0:
            context_unique.append(i)
        else:
            cond = True
            for j in context_unique:
                if i.metadata['Nomor Ketentuan'] == j.metadata['Nomor Ketentuan'] and i.metadata['Ketentuan'] == j.metadata['Ketentuan'] and i.page_content == j.page_content:
                    cond = False
            if cond:
                context_unique.append(i)
    return context_unique

def to_documents(context: list[_DocumentWithState]):
    context_unique = [i.to_document() for i in context]
    return context_unique

vector_store = vector_store_ket
retriever_mmr = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': top_k, 'lambda_mult': 0.25})
retriever_similarity = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': top_k})
self_query_retriever = self_query_retriever_sikepo(
llm_model=llm_model, vector_store=vector_store)
lotr = MergerRetriever(retrievers=[retriever_similarity, retriever_mmr]) 

# remove redundant documents
filter = EmbeddingsRedundantFilter(embeddings=embed_model)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
) | RunnableLambda(to_documents)

In [8]:
compression_retriever.invoke("Siapa yang memiliki Bank?")

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '42/POJK.03/2015', 'Ketentuan': 'Pasal 53 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Konvensional', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Kewajiban Pemenuhan Rasio Kecukupan Likuiditas Liquidity Coverage Ratio Bagi Bank Umum', 'Tanggal Ketentuan': '2015/12/23'}, page_content='3. bank yang dimiliki baik secara sendiri atau bersama-sama oleh warga negara asing dan/atau badan hukum asing kurang dari 50% (lima puluh persen) namun terdapat pengendalian oleh warga negara asing dan/atau badan hukum asing tersebut.\nAyat (2)\nCukup jelas.\nAyat (3)\nCukup jelas.\nAyat (4)\nYang dimaksud dengan kondisi tertentu antara lain apabila Bank dalam kondisi stres.'),
 Document(metadata={'Nomor Ketentuan': '32/POJK.03/2016', 'Ketentuan': 'Pasal 44 ', 'Referensi': '6/POJK.03/2015 Pasal 44 ', 'Kodifikasi Ketentuan': 'Unit Usaha Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Perubahan Atas Peraturan Otoritas Jasa Keuangan Nomor 6POJK032

In [6]:
await compression_retriever.ainvoke("Siapa yang memiliki Bank?")

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '42/POJK.03/2015', 'Ketentuan': 'Pasal 53 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Konvensional', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Kewajiban Pemenuhan Rasio Kecukupan Likuiditas Liquidity Coverage Ratio Bagi Bank Umum', 'Tanggal Ketentuan': '2015/12/23'}, page_content='3. bank yang dimiliki baik secara sendiri atau bersama-sama oleh warga negara asing dan/atau badan hukum asing kurang dari 50% (lima puluh persen) namun terdapat pengendalian oleh warga negara asing dan/atau badan hukum asing tersebut.\nAyat (2)\nCukup jelas.\nAyat (3)\nCukup jelas.\nAyat (4)\nYang dimaksud dengan kondisi tertentu antara lain apabila Bank dalam kondisi stres.'),
 Document(metadata={'Nomor Ketentuan': '19/POJK.03/2014', 'Ketentuan': 'Pasal 11 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Konvensional', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Layanan Keuangan Tanpa Kantor Dalam Rangka Keuangan Inklusif', 'Tanggal Ketentuan

In [24]:
res = lotr.invoke("Siapa yang memiliki Bank?")
res

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berikut: Pasal 2\nBank terdiri dari bank umum konvensional, bank umum syariah, bank perkreditan rakyat, dan/atau bank pembiayaan rakyat syariah.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 2\nCukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '32/POJK.03/2016', 'Ketentuan': 'Pasal 44 ', 'Referensi': '6/POJK.03/2015 Pasal 44 ', 'Kodifikasi Ketentuan': 'Unit Usaha Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Perubahan Atas Peraturan Otoritas Jasa Keuangan Nomor 6POJK032015 Tentang Transparansi dan Publikasi Laporan Bank', 'Tanggal Ketentuan': '2016/08/08'}, page_content='a) P

In [25]:
res = lotr.invoke("Apa peraturan yang harus saya perhatikan terkait dengan QRIS Cross Border?")
res

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '12/POJK.01/2017', 'Ketentuan': 'Pasal 50 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'BPR Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penerapan Program Anti Pencucian Uang dan Pencegahan Pendanaan Terorisme di Sektor Jsa Keuangan', 'Tanggal Ketentuan': '2017/03/16'}, page_content='Nomor Ketentuan: 12/POJK.01/2017\nKetentuan: Pasal 50 \nmemiliki Isi ketentuan sebagai berikut: Pasal 50\nBank Pengirim yang menyediakan jasa Cross Border Correspondent Banking wajib:\na. mendokumentasikan seluruh transaksi Cross Border Correspondent Banking;\nb. menolak untuk berhubungan dan/atau meneruskan hubungan Cross Border Correspondent Banking dengan shell bank; dan\nc. memastikan bahwa Bank Penerima dan/atau Bank Penerus tidak mengijinkan rekeningnya digunakan oleh shell bank pada saat mengadakan hubungan usaha terkait dengan Cross Border Correspondent Banking.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 50\nCukup Jelas'),
 Document

In [18]:
res = await lotr.ainvoke("Siapa yang memiliki Bank?")
res

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berikut: Pasal 2\nBank terdiri dari bank umum konvensional, bank umum syariah, bank perkreditan rakyat, dan/atau bank pembiayaan rakyat syariah.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 2\nCukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'BPR Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berik

In [6]:
res = await lotr.ainvoke("Siapa yang memiliki Bank?")
res

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berikut: Pasal 2\nBank terdiri dari bank umum konvensional, bank umum syariah, bank perkreditan rakyat, dan/atau bank pembiayaan rakyat syariah.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 2\nCukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'BPR Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berik

In [3]:
lotr.invoke("Siapa yang memiliki Bank?")

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

[Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berikut: Pasal 2\nBank terdiri dari bank umum konvensional, bank umum syariah, bank perkreditan rakyat, dan/atau bank pembiayaan rakyat syariah.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 2\nCukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai

In [5]:
await lotr.ainvoke("Siapa yang memiliki Bank?")

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai berikut: Pasal 2\nBank terdiri dari bank umum konvensional, bank umum syariah, bank perkreditan rakyat, dan/atau bank pembiayaan rakyat syariah.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 2\nCukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '36/POJK.03/2019', 'Ketentuan': 'Pasal 2 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penyampaian Laporan Melalui Portal Pelaporan Terintegrasi', 'Tanggal Ketentuan': '2019/12/18'}, page_content='Nomor Ketentuan: 36/POJK.03/2019\nKetentuan: Pasal 2 \nmemiliki Isi ketentuan sebagai

In [8]:
chain.invoke({"question":"Siapa yang bisa memiliki Bank?", "chat_history":""})

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

{'rewrited question': 'Siapa yang dapat memiliki sebuah bank?',
 'context': [Document(metadata={'doc_id': 3105, 'title': 'Surat Edaran Bank Indonesia Nomor 7/7/DPM', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '7/7/DPM/2005', 'effective_date': '2005/03/29', 'file_url': 'https://www.ojk.go.id/Files/regulasi/perbankan/se-bi/2005/lamp2_se_070705.zip', 'page_number': 123}, page_content="II. NON BANK LUAR NEGERI\n1\nBCA FINANCE - HONGKONG\n901\n2\nBEAR STEARNS - LONDON\n902\n3\nBOSTON SAVE DEPOSIT - BOSTON\n903\n4\nBROWN BROTHER HARRIMAN & CO - USA\n904\n5\nCITIBANK LONDON SECURITIES - UK\n905\n6\nCITICORP INVESTMENT BANK - SINGAPORE \n906\n7\nCOUTTS AND CO, LONDON - UNITED KINGDOM\n907\n8\nDEUTSCHE BANK LONDON PRIME BROKERAGE - LONDON\n908\n9\nDEUTSCHE BANK TRUST, CO. - NEW YORK\n909\n10 GE CAPITAL CORPORATION, STANFORD - USA \n911\n11 GK GOH STOCKBROKER PTE. LTD. - SINGAPORE\n912\n12 GOLDMAN SACHS ASIA FINANCE - USA\n913\n13 GSIC - SINGAPORE\n914\n14 K

In [3]:
await chain.ainvoke({"question":"Siapa yang bisa memiliki Bank?", "chat_history":""})

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

{'rewrited question': 'Siapa yang dapat memiliki sebuah bank?',
 'context': [Document(metadata={'doc_id': 2291, 'title': 'Surat Edaran Bank Indonesia Nomor 7/14/DPNP', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '7/14/DPNP/2005', 'effective_date': '2005/04/18', 'file_url': 'https://www.ojk.go.id/Files/regulasi/perbankan/se-bi/2005/lamp_se_071405.pdf', 'page_number': 14}, page_content='Bank A\nBank A\nBank A\nBank A \n \n \n \n(Repo Party)\n(Repo Party)\n(Repo Party)\n(Repo Party) \n \n \n \nPenerbit SSB\nPenerbit SSB\nPenerbit SSB\nPenerbit SSB \n \n \n \n(issuer)\n(issuer)\n(issuer)\n(issuer) \n \n \n \nRepo\nBMPK \nReverse Party \nBMPK \nRepo Party \n2\n\nSource: [7/14/DPNP/2005](https://www.ojk.go.id/Files/regulasi/perbankan/se-bi/2005/lamp_se_071405.pdf#page=14)'),
  Document(metadata={'doc_id': 2009, 'title': 'Surat Edaran Bank Indonesia Nomor 15/4/DPNP', 'sector': None, 'subsector': None, 'regulation_type': None, 'regulation_number': '15/4/DPN

In [12]:
await chain.ainvoke({"question":"Berapa SWDKLLJ dari buldozer?", "chat_history":""})

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '16.78.85.84'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

{'rewrited question': 'Berapa jumlah SWDKLLJ yang harus dibayarkan untuk buldozer?',
 'context': [Document(metadata={'doc_id': 1718, 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '2008/02/26', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf', 'page_number': 4}, page_content='MENTERI KEUANGAN\nREPUBLIK INDONESIA\n-4-\n Mobil penumpang angkutan umum sampai dengan 1600 cc\ng. Bus dan mikro bus bukan angkutan umum sebesar Rp150.000,00\n(seratus lima puluh ribu rupiah).\nh. Bus dan.mikro bus angkutan umum, serta mobil penumpang\nangkutan umum lainnya di atas 1600 cc sebesar Rp87.000,00\n(delapan puluh tujuh ribu rupi